In [2]:
import pandas as pd
import warnings, os

In [3]:
URL = r'https://docs.google.com/spreadsheets/d/1fWtTG28VsCQ2hTvfKPTT2sKMf2Qmhs1hvD-Su3CDHmg/export?output=xlsx'

xl = pd.ExcelFile(URL)
print(xl.sheet_names)

['Linkedin HR', 'Slack', 'Dribbble', 'Behance', 'Fiverr', 'Quora', 'Parser', 'Biteable', 'Upwork', 'Jira', 'Teamwork']


In [5]:
def process_existing_excel(df):
    
    # drop empty columns
    columns_to_drop = [column for column in df.columns if df[column].notna().sum()==0]
    df = df.drop(columns=columns_to_drop)
    
    # set column names to the row values where "No" exists
    df.columns = df[df.iloc[:,0]=="No"].values.flatten().tolist()
    
    # select subtable where first column starts starts with "1"
    index = df[df.iloc[:,0]==1].index
    df = df[df.index>=index[0]].reset_index(drop=True)
    
    # drop empty columns 2
    columns_to_drop2 = [column for column in df.columns if df[column].notna().sum()==0]
    df = df.drop(columns=columns_to_drop2)
    
    # drop groups
    # find name of groups by selecting integers numbers
    df["No"] = df["No"].astype(str).str.strip(' ').apply(lambda x: x[:-1] if x[-1]=='.' else x)
    df["No"] = df["No"].apply(lambda x: x.split(".")[0]+f".{int(x.split('.')[1]):02d}" if "." in x else x)
    df["No"] = df["No"].astype(float)
    
    groups = df[df["No"]%1==0].iloc[:,:2]
    groups = dict(groups.to_dict("split")["data"])
    print("Found next groups", groups)
    df = df[~df["No"].isin(groups.keys())]
    
    df["GROUP"] = df["No"].apply(lambda x: groups[x//1])
    
    # drop empty rows, where second column have Nans
    df = df[df.iloc[:,1].notna()].reset_index(drop=True)
    
    return df

dfs = {}
for name in xl.sheet_names:
    print(name)
    df = pd.read_excel(URL, sheet_name=name)
    try:
        df = process_existing_excel(df)
        dfs.update({name:df})
    except:
        warnings.warn(f"Could not load {name} table")

Linkedin HR
Found next groups {1.0: 'PROFILE', 2.0: 'BUILD NETWORK', 3.0: 'MESSAGES', 4.0: 'CONTENT', 5.0: 'CANDIDATES', 6.0: 'COMPANIES - SUPER LOW'}
Slack
Found next groups {1.0: 'SEARCHING AND REGISTRATION TO NEW SLACK COMMUNITIES ', 2.0: 'INTRODUCTION AND SETTINGS', 3.0: 'MESSAGES', 4.0: 'DOCUMENTATION'}
Dribbble
Found next groups {1.0: 'GET MORE FOLLOWERS', 2.0: 'MESSAGES'}
Behance
Found next groups {1.0: 'GET MORE FOLLOWERS', 2.0: 'MESSAGES'}
Fiverr
Found next groups {1.0: 'MESSAGES', 2.0: 'OFFERS', 3.0: 'REVIEWS', 4.0: 'ORDERS'}
Quora
Found next groups {1.0: 'FIND QUESTIONS AND REPLY', 2.0: nan}
Parser
Found next groups {1.0: 'CONTENT COPY', 2.0: nan}
Biteable
Found next groups {1.0: 'CONTENT ', 2.0: nan}
Upwork
Found next groups {1.0: 'SEARCHING', 2.0: 'PROPOSALS', 3.0: nan, 4.0: nan}
Jira


<ipython-input-5-ac4d9195e132>:44: UserWarning: Could not load Jira table
  warnings.warn(f"Could not load {name} table")


Teamwork


<ipython-input-5-ac4d9195e132>:44: UserWarning: Could not load Teamwork table
  warnings.warn(f"Could not load {name} table")


In [8]:
# Save dataframes to the folder
FOLDER = "data_transformed"
try: 
    os.mkdir(FOLDER) 
except OSError as error: 
    print(error)
    
[df.to_excel(os.path.join(FOLDER,f"{name}.xlsx")) for name,df in dfs.items()]

[Errno 17] File exists: 'data_transformed'


[None, None, None, None, None, None, None, None, None]